In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [23]:
from fnsa.lexicon import get_entries, Lexicon, prepare, set_entries
from fnsa.util import *
import spacy

In [5]:
nlp = spacy.load('en_core_web_sm')
sentences = load_sentences()

In [34]:
lexicon = Lexicon(nlp)

In [35]:
sentence = "TietoEnator was down 1.13 pct to 18.38 , extending recent lows after last week 's second-quarter report , dealers said ."
doc = lexicon(sentence)
show(doc)

 [TietoEnator/xe:org] was [down/dr:-] [1.13/xe:car] pct to [18.38/xe:car] , [extending/dr:+] [recent lows/fe:-] after [last week 's/xe:dat] [second-quarter/xe:dat] report , [dealers/fe:+] said .


In [29]:
sentence = "Initial estimated total value of the contract is $25,000 but it was later to contract."
doc = lexicon(sentence)
show(doc)

 Initial estimated [total value/fe:+] of the [contract/if:lit] is $ [25,000/xe:mon] but it was later to [contract/dr:-] .


In [30]:
sentence = 'Earnings per share ( EPS ) amounted to EUR0 .98 , up from the loss of EUR0 .02 .'
doc = lexicon(sentence)
show(doc, include_text=True)

Earnings per share ( EPS ) amounted to EUR0 .98 , up from the loss of EUR0 .02 .
 [Earnings per share/fe:+] ( [EPS/fe:+] ) amounted to [EUR0/xe:org] .98 , [up/dr:+] from the [loss/fe:-] of [EUR0/xe:org] .02 .


In [32]:
for i, sentence in enumerate(sentences):
    doc = lexicon(sentence)
    show(doc, index=(i+1), include_text=True)
    print("-----")

1. Google expects its EBIT in 2016 to increase 5-10% from 2015.
   [Google/xe:org] expects its [EBIT/fe:+] in [2016/xe:dat] to [increase/dr:+] [5-10%/xe:per] from [2015/xe:dat] .
-----
2. The number of collection errors fell considerably, and operations speeded up.
   The number of [collection errors/fe:-] [fell/dr:-] [considerably/lm:=] , and [operations/fe:+] [speeded up/dr:+] .
-----
3. The Group's operative EBIT will probably be negative.
   The [Group/xe:org] 's operative [EBIT/fe:+] will [probably/if:mod] be [negative/if:rev] .
-----
4. General Electric reports its operating profit increased to $20.5 million.
   [General Electric/xe:org] reports its [operating profit/fe:+] [increased/dr:+] to [$20.5 million/xe:mon] .
-----
5. Ford Motor Company reported higher full-year profits on Thursday boosted by a buoyant period for its car loans unit.
   [Ford Motor Company/xe:org] reported [higher/dr:+] [full-year profits/fe:+] on [Thursday/xe:dat] [boosted/dr:+] by a [buoyant period/fi:+]

## Miscellaneous Stuff - Can be Deleted!

In [ ]:
for lex, category, start, end, index, acceptable in doc.user_data['entries']:
    print("%s --> %s/%s" % (doc[start:end], lex, category))

In [26]:
sentence = "Initial estimated total value of the contract is $25,000 but it always contracts."
doc = lexicon.nlp(sentence)
prepare(doc, lexicon.lexicons)
lexicon.match(doc)
show(doc)
lex_entries = list(get_entries(doc))
lex_entries = sorted(lex_entries, key=lambda item: (-(item[3] - item[2]), item[0]))
#for lex, iid, start, end, index, accepted in lex_entries:
# print(lex, iid, doc[start:end])    
seen = set([])
entries = []
for item in lex_entries:
    lex, iid, start, end, index, accepted = item
    if start in seen or end - 1 in seen: continue
    if lex in lexicon.pos_lexicons:
        match2pos = lexicon.lexicon2matcher[lex].match2pos
        span = doc[start:end]
        phrase = span.text
        if phrase in match2pos:
            pos = " ".join([token.pos_ for token in span])
            if pos != match2pos[phrase]: continue
    seen |= set(range(start, end))
    doc[start]._.lex = lex
    doc[start]._.category = iid
    if lex == EXTRACTED_ENTITY_LEX:
        doc[start]._.index = index
        doc[start]._.accepted = accepted
    entries.append(item)
entries = sorted(entries, key=lambda item: item[2])

set_entries(doc, entries)
delta = 0
for item in entries:
    lex, iid, start, end, index, accepted = item
    start -= delta
    end -= delta
    doc[start:end].merge()
    delta += end - start - 1
if lexicon.prune:
    previous_lex = None
    entries = []
    for token in doc:
        lex = token._.lex
        if lex == DEFAULT_LEX:
            if previous_lex != DEFAULT_LEX: start = token.start
            stop = token.end
        else:
            if previous_lex == DEFAULT_LEX: entries.append((start, stop))
        previous_lex = lex
    delta = 0
    for start, stop in entries:
        start -= delta
        stop -= delta
        doc[start:stop].merge()
        delta += stop - start - 1


print('-----')
for lex, iid, start, end, index, accepted in entries:
    print(lex, iid, doc[start:end])

show(doc)

 Initial estimated total value of the contract is $ 25,000 but it always contracts .
-----
fe + total value of
if lit is
if mod contracts
dr - .
 Initial estimated [total value/fe:+] of the [contract/if:lit] is $ 25,000 but it [always/if:mod] [contracts/dr:-] .


In [ ]:
sentence = "Initial estimated total value of the contract is $25,000 but it always contracts."
match2pos = {'contracts':'VERB'}
doc = lexicon.nlp(sentence)
prepare(doc, lexicon.lexicons)
lexicon.match(doc)
show(doc)
for lex, iid, start, end, index, accepted in doc.user_data['entries']:
    
    if lex == 'dr':
        span = doc[start:end]
        phrase = span.text
        if phrase in match2pos:
            pos = " ".join([token.pos_ for token in span])
            if pos == match2pos[phrase]: print(phrase)
    #print(lex, iid, span, doc[start].pos_)

In [ ]:
doc = self.nlp(text)
prepare(doc, self.lexicons)
for e in doc.ents: 
    store_match(doc, EXTRACTED_ENTITY_LEX, e.label_.lower()[:3], e.start, e.end, -1, e.label_ in ACCEPTABLE_TYPES)
self.match(doc)
self.merge(doc)




In [ ]:
import json
sentence = 'Earnings per share ( EPS ) amounted to EUR0 .98 , up from the loss of EUR0 .02 .'
doc = nlp(sentence)
print(sentence)
prepare(doc, lexicon.lexicons)
print(doc)
#print(json.dumps(doc.user_data, indent=4, sort_keys=True))
#print(doc.user_data)
lexicon.match(doc)
print(doc)
lexicon.merge(doc)
#print(doc)


#print(json.dumps(doc.user_data, indent=4, sort_keys=True))
#print(doc.user_data)

#annotations = annotate(doc)
#print(" ".join(annotations))
#for token in doc: print("[%s/%s:%s]" % (token.text, token._.lex, token._.category))

In [ ]:
??show

In [ ]:
doc.user_data